<a href="https://colab.research.google.com/github/yoooniverse/NLP/blob/main/GloVe%26FastText_practive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.9/974.9 KB 16.5 MB/s eta 0:00:00


use same english data that used in 'EngKorWord2Vec_practice.ipynb'

In [2]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f628f9d29a0>)

In [9]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

parse_text = '\n'.join(target_text.xpath('//content/text()'))   #Imports only the content between <content> and </content> from the xml file.

content_text = re.sub(r'\([^)]*\)', '', parse_text)   #Remove words enclosed in parentheses

In [10]:
print(len(content_text))

24062319


In [12]:
content_text = content_text[:10000000]
print(len(content_text))

10000000


In [13]:
sent_text = sent_tokenize(content_text)

normalized_text = []
for string in sent_text:
  tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
  normalized_text.append(tokens)

result = [word_tokenize(sentence) for sentence in normalized_text]

In [14]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 106649


In [15]:
from glove import Corpus, Glove

corpus = Corpus()

corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [16]:
print(glove.most_similar("man"))

[('woman', 0.961597297518202), ('guy', 0.8732230296963422), ('girl', 0.8639202255853436), ('kid', 0.8602732012718021)]


In [17]:
print(glove.most_similar("boy"))

[('girl', 0.9683533241084212), ('woman', 0.9036302340929019), ('kid', 0.8596524752321857), ('named', 0.8520047620296277)]


In [18]:
print(glove.most_similar("university"))

[('sea', 0.8591755741244471), ('speed', 0.8402555123202957), ('level', 0.8189468183166614), ('harvard', 0.8121668355818998)]


In [19]:
print(glove.most_similar("water"))

[('light', 0.8548915719899662), ('air', 0.8474049689562769), ('food', 0.8350289779049479), ('dna', 0.8333951963267312)]


In [20]:
print(glove.most_similar("physics"))

[('production', 0.9294420410364644), ('economics', 0.920707232885552), ('finance', 0.914222518175187), ('silence', 0.9022584152569584)]


In [21]:
print(glove.most_similar("muscle"))

[('balls', 0.7666712125192937), ('pitch', 0.7554862342800687), ('reward', 0.755009286959137), ('dignity', 0.7444343786131493)]


In [22]:
print(glove.most_similar("clean"))

[('directly', 0.7900752515683187), ('post', 0.7838284581439903), ('warm', 0.7779271454491424), ('heat', 0.774197164991298)]




---



FastText

comapare Word2Vec vs FastText

위에서 전처리한 데이터를 가져와 이전에 했던 word2vec 학습을 진행.('EngKorWord2Vec.ipynb')

In [23]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [25]:
model_result = model.wv.most_similar("electrofishing")

#KeyError: "word 'electrofishing' not in vocabulary"
#Word2Vec cannot calcuate simialrity because it does not have embedding vector for vacabulary that is not studied

KeyError: ignored

FastText

In [26]:
from gensim.models import FastText

model = FastText(result, size=100, window=5, min_count=5, workers=4, sg=1)

In [28]:
model_result = model.wv.most_similar("electrofishing")
print(model_result)

[('electric', 0.8659817576408386), ('electromagnet', 0.8567479848861694), ('electrolux', 0.8557562232017517), ('electron', 0.841770589351654), ('electrical', 0.818234920501709), ('electrons', 0.8171530961990356), ('electrode', 0.8139581680297852), ('electricity', 0.8083945512771606), ('electronic', 0.7816483974456787), ('electrodes', 0.7757441997528076)]


FastText can find similar words and its similarity even though "electrofishing" is not learned.